In [1]:
import psycopg2
from sqlalchemy import create_engine

In [2]:
import psycopg2
from psycopg2 import sql

con = psycopg2.connect(dbname='db_v1',
      user= 'postgres',
       host='',
       port = '5432')


cursor = con.cursor()

# Use the psycopg2.sql module instead of string concatenation 
# in order to avoid sql injection attacks.

In [ ]:
table_query = '''  '''

In [10]:
print("PostgreSQL server information")
print(con.get_dsn_parameters(), "\n")
cursor.execute("SELECT version();")

PostgreSQL server information
{'user': 'postgres', 'passfile': '/home/debian/.pgpass', 'channel_binding': 'prefer', 'dbname': 'db_v1', 'host': '', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 



In [53]:
cursor.execute("SELECT version();")
record = cursor.fetchone()
print("You are connected to - ", record, "\n")

You are connected to -  ('PostgreSQL 13.13 (Debian 13.13-0+deb11u1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',) 



DuplicateTable: relation "test_enerve" already exists


In [10]:
insert_query = ''' INSERT INTO test_enerve (ID, Numero, Mot)
            VALUES (1, 45, 'bonus')''' 
cursor.execute(insert_query)
con.commit()

In [11]:
select_query = ''' select * from test_enerve'''
cursor.execute(select_query)
print(cursor.fetchall())

[(2, 45, 'ENFIN_BORDEL'), (1, 45, 'bonus')]


In [6]:

if (con):
        cursor.close()
        con.close()
        print("PostgreSQL connection is closed")

PostgreSQL connection is closed


sudo -u postgres psql

pg_lsclusters


Ver Cluster Port Status Owner    Data directory              Log file
13  main    5432 online postgres /var/lib/postgresql/13/main /var/log/postgresql/postgresql-13-main.log

(.venv) debian@dev-vm-1683382:~$ createdb ~/projets/data/database/myd_1 -U postgres
(.venv) debian@dev-vm-1683382:~$ createdb ~/projets/data/database/mydb -U postgres

(.venv) debian@dev-vm-1683382:/etc/postgresql/13/main$ psql my_db -U postgres
(.venv) debian@dev-vm-1683382:/etc/postgresql/13/main$ psql -d postgre_1.db -U postgres

Actuellement : commencer à créer les tables via des df pour expérimenter 

+ recup des métadonnées 

In [6]:
import gzip 
import xmltodict 

In [33]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("/")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

In [8]:
p1 = parse_fichier('../../../fichiers20/todo_xml_20/612038.xml.gz')

En tete budget, scellement, en tête budget, et bloc budget

In [36]:
p1['DocumentBudgetaire']['Scellement']

{'@md5': '38EEFB1A871A75E42CE7E98904B62050',
 '@sha1': '636AC74E0042E61EEB9F7C944B2B60CFACAE58C4',
 '@date': '2020-06-24T17:58:52.864+02:00'}

In [72]:
import polars as pl 
import pyarrow as pa 
import glob
import os 
import pandas as pd 

schema_doc_budg = pa.schema([
    pa.field('Id_Fichier', pa.string()),
    pa.field('Nomenclature', pa.string()),
    pa.field('DteStr', pa.date32()),
    pa.field('LibellePoste', pa.string()),
    pa.field('IdPost', pa.string()),
    pa.field('LibelleColl', pa.string()),
    pa.field('IdColl', pa.string()), 
    pa.field('NatCEPL', pa.string()),
    pa.field('date', pa.string()), # ? Pas sûr pour le moment
    pa.field('LibelleEtab', pa.string()),
    pa.field('IdEtab', pa.int64()),
    pa.field('CodColl', pa.int32()),
    pa.field('CodInseeColl', pa.int64()),
    pa.field('CodBud', pa.string()), #A verif
    pa.field('NatDec', pa.int16()),
    pa.field('Exer', pa.int16()),
    pa.field('ProjetBudget', pa.int16()),
    pa.field('DteDec', pa.date32()), 
    pa.field('DteDecEx', pa.date32()), 
    pa.field('NatVote', pa.string()), 
    pa.field('OpeEquip', pa.bool_()),
    pa.field('VoteFormelChap', pa.bool_()),
    pa.field('TypProv', pa.int16()),
    pa.field('BudgPrec', pa.int16()), 
    pa.field('ReprRes', pa.int16()),
    pa.field('NatFonc', pa.int16()),
    pa.field('CodTypBud', pa.string())
    ])

In [85]:
def extraction_donnees(chemin) : 
 dict_annexe = {}
 for a, b in chemin.items() : 
   dict_annexe.update({a : b.get('@V')})
 return dict_annexe 

In [153]:
def extraction_ligne_budget(chemin_annexe, dict_metadonnees) : 
 liste_annexe = []
 for row in chemin_annexe : 
  liste_par_ligne = {}
  for a, b in row.items() : 
   liste_par_ligne.update({a : b.get('@V')})
   liste_par_ligne.update(dict_metadonnees)
  liste_annexe.append(liste_par_ligne)
 return liste_annexe 

In [162]:
v2 = parse_fichier('../../../fichiers20/todo_xml_20/t10/612013.xml.gz')
v23 = v2['DocumentBudgetaire']['Budget']['LigneBudget']
#dicottt = extraction_donnees(v22)

Nature 60632
ContNat 011
ArtSpe false
CodRD D
MtBudgPrec 11962.00
MtPropNouv 19335.00
MtPrev 19335.00
CredOuv 19335.00
OpBudg 0
MtSup 11962.00
CaracSup F
Nature 6332
ContNat 012
ArtSpe false
CodRD D
MtBudgPrec 450.00
MtPropNouv 500.00
MtPrev 500.00
CredOuv 500.00
OpBudg 0
MtSup 450.00
CaracSup F
Nature 6411
ContNat 012
ArtSpe false
CodRD D
MtBudgPrec 365000.00
MtPropNouv 352000.00
MtPrev 352000.00
CredOuv 352000.00
OpBudg 0
MtSup 365000.00
CaracSup F
Nature 74834
ArtSpe false
CodRD R
MtBudgPrec 4104.00
MtPropNouv 4084.00
MtPrev 4084.00
CredOuv 4084.00
OpBudg 0
MtSup 4104.00
CaracSup F
Nature 6331
ContNat 012
ArtSpe false
CodRD D
MtBudgPrec 4250.00
MtPropNouv 5200.00
MtPrev 5200.00
CredOuv 5200.00
OpBudg 0
MtSup 4250.00
CaracSup F
Nature 6454
ContNat 012
ArtSpe false
CodRD D
MtBudgPrec 4000.00
MtPropNouv 4500.00
MtPrev 4500.00
CredOuv 4500.00
OpBudg 0
MtSup 4000.00
CaracSup F
Nature 6451
ContNat 012
ArtSpe false
CodRD D
MtBudgPrec 66000.00
MtPropNouv 72000.00
MtPrev 72000.00
CredOuv 720

AttributeError: 'list' object has no attribute 'get'

In [199]:
df_concours = pd.read_parquet('../../../data/infos_siren_siret_cog_collectivites.parquet')

df_concours.head(1)

,siren,statutDiffusionUniteLegale,unitePurgeeUniteLegale,dateCreationUniteLegale,sigleUniteLegale,sexeUniteLegale,prenom1UniteLegale,prenom2UniteLegale,prenom3UniteLegale,prenom4UniteLegale,prenomUsuelUniteLegale,pseudonymeUniteLegale,identifiantAssociationUniteLegale,trancheEffectifsUniteLegale,anneeEffectifsUniteLegale,dateDernierTraitementUniteLegale,nombrePeriodesUniteLegale,categorieEntreprise,anneeCategorieEntreprise,dateDebut,etatAdministratifUniteLegale,nomUniteLegale,nomUsageUniteLegale,denominationUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,denominationUsuelle3UniteLegale,categorieJuridiqueUniteLegale,activitePrincipaleUniteLegale,nomenclatureActivitePrincipaleUniteLegale,nicSiegeUniteLegale,economieSocialeSolidaireUniteLegale,societeMissionUniteLegale,caractereEmployeurUniteLegale,categorieJuridiqueUniteLegale:1,libelleCategorieJuridique,siren:1,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,nombrePeriodesEtablissement,complementAdresseEtablissement,numeroVoieEtablissement,indiceRepetitionEtablissement,...,libelleVoieEtablissement,codePostalEtablissement,libelleCommuneEtablissement,libelleCommuneEtrangerEtablissement,distributionSpecialeEtablissement,codeCommuneEtablissement,codeCedexEtablissement,libelleCedexEtablissement,codePaysEtrangerEtablissement,libellePaysEtrangerEtablissement,complementAdresse2Etablissement,numeroVoie2Etablissement,indiceRepetition2Etablissement,typeVoie2Etablissement,libelleVoie2Etablissement,codePostal2Etablissement,libelleCommune2Etablissement,libelleCommuneEtranger2Etablissement,distributionSpeciale2Etablissement,codeCommune2Etablissement,codeCedex2Etablissement,libelleCedex2Etablissement,codePaysEtranger2Etablissement,libellePaysEtranger2Etablissement,dateDebut:1,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,CODGEO,LIBGEO,DEP,REG,EPCI,NATURE_EPCI,ARR,CV,ZE2020,UU2020,TUU2017,TDUU2017,AAV2020,TAAV2017,TDAAV2017,CATEAAV2020,BV2022
0,130001589,O,None,2005-12-25,CDAD,None,None,None,None,None,None,None,None,02,2021.0,2023-11-30T10:17:02,4,PME,2021.0,2016-04-01,A,None,None,CONSEIL DEP DE L'ACCES AU DROIT,None,None,None,7410,84.23Z,NAFRev2,22,None,None,O,7410,Groupement d'intérêt public (GIP),130001589,14,13000158900014,O,2005-12-25,NN,NaN,None,2019-11-14T14:00:12,False,4,TRIBUNAL DE GRANDE INSTANCE,4,None,...,DU PALAIS,01000,BOURG-EN-BRESSE,None,BP 306,01053,01011,BOURG EN BRESSE CEDEX,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2016-04-01,F,None,None,None,None,84.23Z,NAFRev2,O,01053,Bourg-en-Bresse,01,84,200071751,CA,012,0199,8405,01501,5,51,080,2,24,11,01053


In [203]:
df_concours['categorieJuridiqueUniteLegale'].value_counts().head(5)

categorieJuridiqueUniteLegale
7210    189088
7361     46619
7353     24790
7346     19625
7364     15670
Name: count, dtype: int64

In [ ]:
creation_table_etablissement = ''' 
 siren INTEGER, 
 siret INTEGER,
 codeCommuneEtablissement VARCHAR(32), 
 CODGEO VARCHAR(32), 
 LIBGEO VARCHAR(32), 
 DEP VARCHAR(32),
 REG VARCHAR(32),

 EPCI VARCHAR(32),
 
 libelleCommuneEtablissement VARCHAR(32),
 denominationUniteLegale VARCHAR(32),  
 libelleCategorieJuridique VARCHAR(32),
 
 categorieJuridiqueUniteLegale INTEGER, 
 sigleUniteLegale VARCHAR(32), 
 dateCreationUniteLegale DATE,
 

'''

In [ ]:
creation_table_charge = ''' CREATE TABLE Document_Budgetaire (
 Id_Fichier INTEGER, 
 Exer INTEGER,
 CodTypeCharge VARCHAR(32), #code 01 ou 02
 NatDepTransf VARCHAR(32),
 DureeEtal INTEGER, 
 DtDelib DATE,
 MtDepTransf FLOAT(32),
 MtAmort FLOAT(32),
 MtDotAmort FLOAT(32), 
 Champ_Editeur VARCHAR(32), #Toujours bizarre
 )
''' 

In [ ]:
creatiopn_table_concours =  '''CREATE TABLE Document_Budgetaire ( 
 Id_Fichier INTEGER,
 CodNatJurBenefCA VARCHAR(32), 
 Exer INTEGER, 
 NatDec VARCHAR(32), 
 LibOrgaBenef VARCHAR(32),
 MtSubv FLOAT(32), 
 CodArticle VARCHAR(32), #Numero mais avec des tirets
 CodInvFonc VARCHAR(32), #F et I 
 LibPrestaNat VARCHAR(32), 
 DenomOuNumSubv VARCHAR(32),  
 Champ_Editeur VARCHAR(32), #Contient des [B_Subventions.IdSubvention=6838]
 ObjSubv VARCHAR(32), 
 Siret VARCHAR(32), #pas tous remplis 
 PopCommune INTEGER, #pas sûr que ce soit bon 
 
)
''' 

In [145]:
creation_table_doc_budgetaire = ''' 
 CREATE TABLE Document_Budgetaire (
 Id_Fichier INTEGER PRIMARY KEY NOT NULL, 
 Nomenclature VARCHAR(32), 
 Exer INTEGER, 
 IdColl VARCHAR(32),
 CodColl VARCHAR(32),
 
 DteStr DATE,
 @date ,                #Timestamp ?
 DteDec DATE,
 DteDecEx DATE, 

 LibellePoste VARCHAR(32),
 LibelleColl VARCHAR(32), #poubelle 
 NatCEPL VARCHAR(32), #poubelle
 Departement VARCHAR(32), #poubelle
 LibelleEtab VARCHAR(32), #poubelle
 IdEtab INTEGER, 
 NatDec INTEGER, 
 NatVote VARCHAR(32),
 OpeEquip BOOLEAN, 
 CodInseeColl VARCHAR(32), #Contient des Z dans les codes 

 ProjetBudget INTEGER, 
 OpeEquip BOOLEAN,
 VoteFormelChap BOOLEAN, 
 TypProv INTEGER, 
 BudgPrec INTEGER, 
 RefProv VARCHAR(32), #Colonne étrange
 ReprRes INTEGER, 
 NatFonc INTEGER, 
 PresentationSimplifiee BOOLEAN, #besoin nettoyage
 DepFoncN2 FLOAT(32), 
 RecFoncN2 FLOAT(32),
 DepInvN2 FLOAT(32),
 RecInvN2 FLOAT(32),
 CodTypBud VARCHAR(32), 
 IdPost VARCHAR(32), 
 CodBud VARCHAR(32), # 0 en début 
 @md5 VARCHAR(32), #poubelle
 @sha1 VARCHAR(32), #poubelle
 )

'''